In [2]:
import torch
torch.set_grad_enabled(False)  # Disable gradient calculation for inference

In [3]:
import sys

sys.path.append('/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/repos/segfm3d-nora')
sys.path.append('/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/repos/nnunetv2')

In [4]:
from scipy import integrate
from src.eval_metrics import (  # TODO : Use the competition repo as source instead
    compute_edt,
    compute_multi_class_dsc,
    compute_multi_class_nsd,
    sample_coord,
)
import cc3d

In [5]:
config = {
    "DATA_DIR": "/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data",
    "RESULTS_DIR": "/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/results",
    "SAM_CKPT_PATH": "/nfs/norasys/notebooks/camaret/SAM-Med3D/ckpt/sam_med3d_turbo_bbox_cvpr.pth",
    "SAM_REPO_DIR": "/nfs/norasys/notebooks/camaret/SAM-Med3D",
    "ONNX_MODEL_PATH": "/nfs/norasys/notebooks/camaret/model_inference/models/sammed3d.onnx",
    "NNINT_CKPT_DIR": "/nfs/norasys/notebooks/camaret/model_checkpoints/nnint",
}

In [6]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import time
import sys
import numpy as np
import torch
import contextlib
import gc
from nnInteractive.inference.inference_session import nnInteractiveInferenceSession
import os.path


In [8]:
args_for_network = {
    "arch_class_name": "dynamic_network_architectures.architectures.unet.ResidualEncoderUNet",
    "arch_kwargs": {
        "n_stages": 6,
        "features_per_stage": [32, 64, 128, 256, 320, 320],
        "conv_op": "torch.nn.modules.conv.Conv3d",
        "kernel_sizes": [
            [3, 3, 3],
            [3, 3, 3],
            [3, 3, 3],
            [3, 3, 3],
            [3, 3, 3],
            [3, 3, 3],
        ],
        "strides": [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2]],
        "n_blocks_per_stage": [1, 3, 4, 6, 6, 6],
        "n_conv_per_stage_decoder": [1, 1, 1, 1, 1],
        "conv_bias": True,
        "norm_op": "torch.nn.modules.instancenorm.InstanceNorm3d",
        "norm_op_kwargs": {"eps": 1e-05, "affine": True},
        "dropout_op": None,
        "dropout_op_kwargs": None,
        "nonlin": "torch.nn.LeakyReLU",
        "nonlin_kwargs": {"inplace": True},
    },
    "arch_kwargs_req_import": ["conv_op", "norm_op", "dropout_op", "nonlin"],
    "input_channels": 8,
    "output_channels": 2,
    "allow_init": True,
    "deep_supervision": True,
}

In [12]:
from nnunetv2.utilities.get_network_from_plans import get_network_from_plans
network = get_network_from_plans(**args_for_network).cuda()
orig_checkpoint_dir = '/nfs/norasys/notebooks/camaret/model_checkpoints/nnint/nnInteractive_v1.0/'
newer_checkpoint_dir = "/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/data/model-checkpoints/try_from_nb_less_augment/"
if os.path.exists(newer_checkpoint_dir):
    chkpt_dict = torch.load(os.path.join(newer_checkpoint_dir, 'fold_0/checkpoint_final.pth'), weights_only=False)
else:
    chkpt_dict = torch.load(os.path.join(orig_checkpoint_dir, 'fold_0/checkpoint_final.pth'), weights_only=False)

network.load_state_dict(chkpt_dict['network_weights'])

<All keys matched successfully>

In [13]:
%aimport src.training.transforms

In [14]:
from batchgeneratorsv2.transforms.base.basic_transform import BasicTransform
from batchgeneratorsv2.transforms.intensity.brightness import (
    MultiplicativeBrightnessTransform,
)
from batchgeneratorsv2.transforms.intensity.contrast import (
    ContrastTransform,
    BGContrast,
)
from batchgeneratorsv2.transforms.intensity.gamma import GammaTransform
from batchgeneratorsv2.transforms.intensity.gaussian_noise import GaussianNoiseTransform
from batchgeneratorsv2.transforms.nnunet.random_binary_operator import (
    ApplyRandomBinaryOperatorTransform,
)
from batchgeneratorsv2.transforms.nnunet.remove_connected_components import (
    RemoveRandomConnectedComponentFromOneHotEncodingTransform,
)
from batchgeneratorsv2.transforms.nnunet.seg_to_onehot import (
    MoveSegAsOneHotToDataTransform,
)
from batchgeneratorsv2.transforms.noise.gaussian_blur import GaussianBlurTransform
from batchgeneratorsv2.transforms.spatial.low_resolution import (
    SimulateLowResolutionTransform,
)
from batchgeneratorsv2.transforms.spatial.mirroring import MirrorTransform
from batchgeneratorsv2.transforms.spatial.spatial import SpatialTransform
from batchgeneratorsv2.transforms.utils.compose import ComposeTransforms
from batchgeneratorsv2.transforms.utils.deep_supervision_downsampling import (
    DownsampleSegForDSTransform,
)
from batchgeneratorsv2.transforms.utils.nnunet_masking import MaskImageTransform
from batchgeneratorsv2.transforms.utils.pseudo2d import (
    Convert3DTo2DTransform,
    Convert2DTo3DTransform,
)
from batchgeneratorsv2.transforms.utils.random import RandomTransform
from batchgeneratorsv2.transforms.utils.remove_label import RemoveLabelTansform
from batchgeneratorsv2.transforms.utils.seg_to_regions import (
    ConvertSegmentationToRegionsTransform,
)
from src.training.transforms import AddSegToImageTransform
from src.training.transforms import NormalizeSingleImageTransform
from src.training.transforms import AddBBoxAndEmptyChannelsSingleClassTransform
from src.training.transforms import MONAIRandSpatialTransform
from src.training.transforms import MONAIFixedSpatialTransform


only_2d_bbox = False

test_transform = ComposeTransforms(
    [
        NormalizeSingleImageTransform(),
        MONAIFixedSpatialTransform(),
        RemoveLabelTansform(segmentation_channels=None, label_value=-1, set_to=0),
        AddBBoxAndEmptyChannelsSingleClassTransform(only_2d_bbox=only_2d_bbox),
        AddSegToImageTransform(),
    ]
)

# let's make very little augmentation
train_transform = ComposeTransforms(
    [
        NormalizeSingleImageTransform(),
        MONAIRandSpatialTransform(
            rotate_range=(0.025 * np.pi * 2, 0.025 * np.pi * 2, 0.025 * np.pi * 2),
            scale_range=(0.2, 0.2, 0.2),
            prob_affine=0.5,
        ),
        # RandomTransform(
        #     apply_probability=0.1,
        #     transform=GaussianNoiseTransform(
        #         noise_variance=(0, 0.1), p_per_channel=1, synchronize_channels=True
        #     ),
        # ),
        # RandomTransform(
        #     apply_probability=0.2,
        #     transform=GaussianBlurTransform(
        #         blur_sigma=(0.5, 1.0),
        #         benchmark=True,
        #         synchronize_channels=False,
        #         synchronize_axes=False,
        #         p_per_channel=0.5,
        #     ),
        # ),
        # MirrorTransform(allowed_axes=(0, 1, 2)),
        RemoveLabelTansform(segmentation_channels=None, label_value=-1, set_to=0),
        DownsampleSegForDSTransform(
            ds_scales=[
                [1.0, 1.0, 1.0],
                [0.5, 0.5, 0.5],
                [0.25, 0.25, 0.25],
                [0.125, 0.125, 0.125],
                [0.0625, 0.0625, 0.0625],
            ]
        ),
        AddBBoxAndEmptyChannelsSingleClassTransform(only_2d_bbox=only_2d_bbox),
        AddSegToImageTransform(),
    ]
)

Initializing AddBBoxAndEmptyChannelsSingleClassTransform
Initializing AddSegToImageTransform
Initializing AddBBoxAndEmptyChannelsSingleClassTransform
Initializing AddSegToImageTransform


In [15]:
from src.training.npzdataset import NPZPytorchDataset, NPZDataset
from src.training.utils import fast_ids_to_deterministic_floats
import numpy as np

from glob import glob
parent_folder = '/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data/3D_train_npz_random_10percent_16G/'
parent_save_folder = '/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_incontext/SegFM3D/data/own-preproc/'

all_npz_file_paths = glob(os.path.join(parent_folder, '**/*.npz'), recursive=True)
to_exclude = ['Ultrasound/US_Low-limb-Leg/US_Low-limb-Leg07.npz',
       'Ultrasound/US_Low-limb-Leg/US_Low-limb-Leg24.npz',
       'Ultrasound/US_Low-limb-Leg/US_Low-limb-Leg37.npz',
       'CT/CT_Abdomen1K/CT_Abdomen1K_Case_00931.npz',
       'Microscopy/Microscopy_cremi/Microscopy_cremi_002_sc.npz',
       'Microscopy/Microscopy_cremi/Microscopy_cremi_000_sc.npz',
       'Microscopy/Microscopy_cremi/Microscopy_cremi_001_sc.npz']


identifiers = np.array([f.replace(parent_folder, "") for f in all_npz_file_paths])
identifiers = np.array([f for f in identifiers if f not in to_exclude])

train_mask = fast_ids_to_deterministic_floats(identifiers) < 0.8
train_files = identifiers[train_mask]
test_files = identifiers[~train_mask]
dataset_tr = NPZDataset(parent_folder, parent_save_folder, train_files)
dataset_val = NPZDataset(parent_folder, parent_save_folder, test_files)
train_set = NPZPytorchDataset(
    dataset_tr, transform=train_transform)

test_set = NPZPytorchDataset(
    dataset_val, transform=test_transform)

In [16]:
n_jobs = int(os.environ['SLURM_CPUS_PER_TASK'])

In [17]:
import torch
train_loader = torch.utils.data.DataLoader(
    train_set,
    shuffle=True,
    drop_last=True,
    num_workers=n_jobs,
    batch_size=2,
)

In [18]:
test_set = NPZPytorchDataset(
    dataset_val, transform=test_transform)
test_loader = torch.utils.data.DataLoader(
    test_set,
    shuffle=False,
    drop_last=False,
    num_workers=n_jobs,
    batch_size=2,
)

In [19]:
from tqdm.autonotebook import tqdm, trange
import torch
from src.training.model_wrap import ModelPrevSegAndClickWrapper
from src.training.utils import defer_keysignal_with_grad

wrapped_network = ModelPrevSegAndClickWrapper(network, n_max_clicks=1)

optim_network = torch.optim.AdamW(
    wrapped_network.parameters(), lr=3e-4, weight_decay=1e-5
)

with custom forward print shape


In [20]:
from copy import deepcopy
import shutil
out_dir = newer_checkpoint_dir#'/nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/data/model-checkpoints/try_from_nb_all_data_competition_bbox/'
try:
    shutil.copytree(orig_checkpoint_dir, out_dir)
except FileExistsError:
    print(f"path {out_dir} exists already, fine")
trained_chkpt_path = os.path.join(out_dir, 'fold_0/checkpoint_final.pth')
assert os.path.exists(trained_chkpt_path)


trained_chkpt = deepcopy(chkpt_dict)

path /nfs/data/nii/data1/Analysis/GPUnet/ANALYSIS_segfm-robin/data/model-checkpoints/try_from_nb_less_augment/ exists already, fine


In [22]:
from src.training.utils import compute_binary_dsc
from IPython.display import display
import pandas as pd
n_epochs = 25
moving_dsc = None
all_mean_dscs = []
for i_epoch in trange(n_epochs):
    epoch_mean_dscs = []
    for batch in (pbar := tqdm(train_loader)):
        with defer_keysignal_with_grad():
            targets = batch["segmentation"][0][:,0].long().cuda()
            out = wrapped_network(batch["image"].cuda())
            # assuming deep supervision
            pred = out[0]
            assert pred[:,0].shape == targets.shape
            cent = torch.nn.functional.cross_entropy(pred, targets)
            binary_pred = torch.sigmoid(torch.diff(pred, dim=1)[:,0])
            assert binary_pred.shape == targets.shape
            eps = 1e-3
            intersect = torch.sum(binary_pred * targets, dim=(1,2,3))
            sum_pred = torch.sum(binary_pred, dim=(1,2,3))
            sum_targets = torch.sum(targets, dim=(1,2,3))
            mean_dsc = torch.mean((2 * intersect + eps) / (sum_pred + sum_targets + eps))
            loss = cent + 10 * (1 - mean_dsc)
            optim_network.zero_grad()
            loss.backward()
            if torch.isfinite(loss).item():
                optim_network.step()
            else:
                print("loss not finite, not updating!")
            optim_network.zero_grad()
        epoch_mean_dscs.append(mean_dsc.item())
        if torch.isfinite(loss).item():
            moving_dsc = mean_dsc if moving_dsc is None else (moving_dsc * 0.98 + mean_dsc * 0.02)
        pbar.set_postfix(dict(moving_dsc=moving_dsc.item()))
        print(mean_dsc.item())
        print(cent.item())
        print()
    print("mean dsc", np.nanmean(epoch_mean_dscs))
    all_mean_dscs.extend(epoch_mean_dscs)
    
    print(f"Epoch {i_epoch}")
    if torch.isfinite(loss).item():
        trained_chkpt['network_weights'] = wrapped_network.orig_network.state_dict()
        torch.save(trained_chkpt, trained_chkpt_path)
        print(f"Checkpoint saved to {trained_chkpt_path}")

    test_results = []
    for batch in (pbar := tqdm(test_loader)):
        targets = batch["segmentation"][:, 0].long().cuda()
        out = wrapped_network(batch["image"].cuda())
        # assuming deep supervision
        pred = out[0]
        assert pred[:, 0].shape == targets.shape
        cent = torch.nn.functional.cross_entropy(pred, targets)
        binary_pred = torch.sigmoid(torch.diff(pred, dim=1)[:, 0])
        assert binary_pred.shape == targets.shape
        eps = 1e-3
        intersect = torch.sum(binary_pred * targets, dim=(1, 2, 3))
        sum_pred = torch.sum(binary_pred, dim=(1, 2, 3))
        sum_targets = torch.sum(targets, dim=(1, 2, 3))
        mean_dsc = torch.mean((2 * intersect + eps) / (sum_pred + sum_targets + eps))
        test_results.append(
            dict(
                hard_dsc=compute_binary_dsc((binary_pred > 0.5), targets),
                soft_dsc=((2 * intersect + eps) / (sum_pred + sum_targets + eps)).cpu().numpy().mean(),
                cent=cent.item()
            )
        )
    display(pd.DataFrame(test_results).mean())

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1197 [00:00<?, ?it/s]

0.07295124232769012
0.32169309258461

0.6658397912979126
0.025175178423523903

0.6257704496383667
0.06659174710512161

0.3220725357532501
0.8085110187530518

0.30166733264923096
0.016549095511436462

0.8189313411712646
0.00448513962328434

0.6596202850341797
0.07324261218309402

0.2904227077960968
0.1171383410692215

0.46019282937049866
0.006461052224040031

0.6236398816108704
0.111236572265625

0.7216601967811584
0.002077145269140601

0.7687251567840576
0.0073218196630477905

0.8172993659973145
0.003733764635398984

0.8272097110748291
0.002005561487749219

0.6323584318161011
0.06381910294294357

0.6365022659301758
0.050886232405900955

0.5274644494056702
0.37567275762557983

0.8171185255050659
0.04159553349018097

0.8111063241958618
0.06003205105662346

0.6033905744552612
0.07994028925895691

0.259347528219223
0.04981687292456627

0.4240840673446655
0.0016762728337198496

0.0025926106609404087
0.005019864067435265

0.3969046473503113
0.00710774352774024

0.16366645693778992
0.03485777

  0%|          | 0/292 [00:00<?, ?it/s]

hard_dsc    0.640888
soft_dsc    0.637585
cent        0.042465
dtype: float64

  0%|          | 0/1197 [00:00<?, ?it/s]

0.7956713438034058
0.00046719159581698477

0.6868690252304077
0.0464128777384758

0.5950891375541687
0.007448619231581688

0.7610576152801514
0.2248038947582245

0.6730836033821106
0.006336606107652187

0.18263974785804749
0.3722342848777771

0.8363175392150879
0.012208073399960995

0.8245724439620972
0.006185085047036409

0.7534611225128174
0.001830944325774908

0.8019264936447144
0.0026786834932863712

0.7868896722793579
0.01008562371134758

0.6672244071960449
0.022863829508423805

0.46542972326278687
0.021977806463837624

0.3066227436065674
0.029329469427466393

0.501090407371521
0.0010293618543073535

0.793439507484436
0.00750220101326704

0.8194817304611206
0.010175341740250587

0.49019330739974976
0.03503163158893585

0.5446739196777344
0.030694447457790375

0.7398455142974854
0.04748520627617836

0.7771064639091492
0.00349326035939157

0.4755467474460602
0.0018714896868914366

0.7637421488761902
0.024292360991239548

0.6782764196395874
0.056141920387744904

0.7277601361274719
0.

In [ ]:
# now try click changes, better sampling, gradient accum?
# different sampling?

In [45]:
test_results = []
for batch in (pbar := tqdm(test_loader)):
    targets = batch["segmentation"][:, 0].long().cuda()
    out = wrapped_network(batch["image"].cuda())
    # assuming deep supervision
    pred = out[0]
    assert pred[:, 0].shape == targets.shape
    cent = torch.nn.functional.cross_entropy(pred, targets)
    binary_pred = torch.sigmoid(torch.diff(pred, dim=1)[:, 0])
    assert binary_pred.shape == targets.shape
    eps = 1e-3
    intersect = torch.sum(binary_pred * targets, dim=(1, 2, 3))
    sum_pred = torch.sum(binary_pred, dim=(1, 2, 3))
    sum_targets = torch.sum(targets, dim=(1, 2, 3))
    mean_dsc = torch.mean((2 * intersect + eps) / (sum_pred + sum_targets + eps))
    test_results.append(
        dict(
            hard_dsc=compute_binary_dsc((binary_pred > 0.5), targets),
            soft_dsc=((2 * intersect + eps) / (sum_pred + sum_targets + eps)).cpu().numpy().mean(),
            cent=cent.item()
        )
    )

  0%|          | 0/179 [00:00<?, ?it/s]

In [46]:
display(pd.DataFrame(test_results).mean())

hard_dsc    0.788758
soft_dsc    0.788639
cent        0.020879
dtype: float64

In [35]:
1

In [36]:
1

hard_dsc    0.695916
soft_dsc    0.696083
cent        0.030052
dtype: float64

0.7422285987565758

array([0.7661771, 0.6034883], dtype=float32)